In [ ]:
'''
A Reccurent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [ ]:
# Import MINST data
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np

In [2]:
'''
To classify images using a reccurent neural network, we consider every image row as a sequence of pixels.
Because MNIST image shape is 28*28px, we will then handle 28 sequences of 28 steps for every sample.
'''

# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 23 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

In [3]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
istate = tf.placeholder(tf.float32, [None, 2*n_hidden]) #state & cell => 2x n_hidden
y = tf.placeholder(tf.float32, [None, n_classes])

# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [4]:
istate.get_shape()

TensorShape([Dimension(None), Dimension(46)])

In [5]:
def RNN(_X, _istate, _weights, _biases):
    print _X.get_shape()
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    print _X.get_shape()
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) # (n_steps*batch_size, n_input)
    print _X.get_shape()
    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden']
    print _X.get_shape()
    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, n_steps, _X) # n_steps * (batch_size, n_hidden)
#     print _X.get_shape()
    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, _X, initial_state=_istate)
    
    # Linear activation
    # Get inner loop last output
    return (tf.matmul(outputs[-1], _weights['out']) + _biases['out']), _X,outputs, states

In [6]:
pred, _X,o, s = RNN(x, istate, weights, biases)

TensorShape([Dimension(None), Dimension(28), Dimension(28)])
TensorShape([Dimension(28), Dimension(None), Dimension(28)])
TensorShape([Dimension(None), Dimension(28)])
TensorShape([Dimension(None), Dimension(23)])


In [8]:
len(_X), _X[0].get_shape()

(28, TensorShape([Dimension(None), Dimension(23)]))

In [7]:
pred.get_shape()

TensorShape([Dimension(None), Dimension(10)])

In [9]:
len(o), o[0].get_shape(), 

(28, TensorShape([Dimension(None), Dimension(23)]))

In [10]:
len(s),s[0].get_shape()

(28, TensorShape([Dimension(None), Dimension(46)]))

In [11]:
# pred = RNN(x, istate, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_xs = batch_xs.reshape((batch_size, n_steps, n_input))
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys,
                                       istate: np.zeros((batch_size, 2*n_hidden))})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys,
                                                istate: np.zeros((batch_size, 2*n_hidden))})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys,
                                             istate: np.zeros((batch_size, 2*n_hidden))})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                  ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    # Calculate accuracy for 256 mnist test images
    test_len = 256
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                                             istate: np.zeros((test_len, 2*n_hidden))})

Iter 1280, Minibatch Loss= 3.967579, Training Accuracy= 0.09375
Iter 2560, Minibatch Loss= 2.996217, Training Accuracy= 0.11719
Iter 3840, Minibatch Loss= 2.431879, Training Accuracy= 0.14062
Iter 5120, Minibatch Loss= 2.482943, Training Accuracy= 0.14844
Iter 6400, Minibatch Loss= 2.223346, Training Accuracy= 0.21094
Iter 7680, Minibatch Loss= 2.246519, Training Accuracy= 0.15625
Iter 8960, Minibatch Loss= 2.138725, Training Accuracy= 0.21875
Iter 10240, Minibatch Loss= 1.993708, Training Accuracy= 0.32031
Iter 11520, Minibatch Loss= 1.911527, Training Accuracy= 0.35156
Iter 12800, Minibatch Loss= 2.006713, Training Accuracy= 0.26562
Iter 14080, Minibatch Loss= 1.923802, Training Accuracy= 0.32031
Iter 15360, Minibatch Loss= 1.860279, Training Accuracy= 0.35938
Iter 16640, Minibatch Loss= 1.756133, Training Accuracy= 0.37500
Iter 17920, Minibatch Loss= 1.570304, Training Accuracy= 0.43750
Iter 19200, Minibatch Loss= 1.470616, Training Accuracy= 0.52344
Iter 20480, Minibatch Loss= 1.31

In [13]:
sess.close()